# TODO:

- [ ] Global metrics

In [1]:
# dataset interface should be {'train', 'dev', 'test'?, 'X_column', 'label_column'}
import pandas as pd
from hamison_datasets.UPV.load_upv import load_upv_dataset

upv = load_upv_dataset()
# upv_notmapped = load_upv_dataset(map_values=False)

ds = upv[upv['text'].notnull()]
campañas = upv['¿Qué campaña?'].unique().tolist()
campañas_order = {v: k for k, v in enumerate(campañas)}

upv_features = ['1. Espacio',
                '2. Texto', '3. Multimedia', '4. Elemento desinformador',
                '5.  Tipo multimedia', '6. Alteración multimedia',
                '7. Cuenta emisora', '8. Fuente',
                '9. Protagonista', '10. Atributo',
                '11. Macro tema', '12. Populismo',
                '13. Ataque', '14. Tipo de ataque']
upv_features = [c for c in upv.columns if '.' in c]
# for feat in upv_features:
#     print(upv[feat].value_counts(), '\nTotal:',
#           upv[(upv['text'].notnull() & upv[feat].notnull())].shape[0], '\n')
upv.columns

Loaded UPV dataset from UPV_allURLs.csv
Loading html files...
Removing rows without html: [6668]


Extracting articles from HTML:   0%|          | 0/408 [00:00<?, ?it/s]

Index(['Medio', 'Título desmentido', 'URL', 'Titular enlazado',
       '¿Periodo de campaña?', '¿Qué campaña?', '¿Campaña?', 'Único',
       'Duplicado misma campaña', 'Corpus', 'Numeración duplicados',
       'ID Corpus', 'ID ALE', '1. Espacio', '2. Texto', '3. Multimedia',
       '4. Elemento desinformador', '5.  Tipo multimedia',
       '6. Alteración multimedia', '7. Cuenta emisora', '8. Fuente',
       '9. Protagonista', '10. Atributo', '11. Macro tema', '12. Populismo',
       '13. Ataque', '14. Tipo de ataque', '5.  Tipo multimedia reduced',
       '7. Cuenta emisora reduced', '9. Protagonista reduced',
       '14. Tipo de ataque reduced', 'html', 'article', 'text'],
      dtype='object')

In [4]:
# load GPT preds
import pandas as pd


preds_gpt_9 = pd.read_json(
    'results/results_gpt-4o-mini_9.json'
).set_index('ID_Medio_Crono').rename(
    columns={'prediction': 'prediction_9', 'reasoning': 'reasoning_9'}
)

preds_gpt_1112 = pd.read_json(
    'results_gpt-4o-mini_11-12.json'
).set_index('ID_Medio_Crono').rename(
    columns={'theme': 'prediction_11', 'theme_reasoning': 'reasoning_11',
             'populist_strategy': 'prediction_12', 'populist_strategy_reasoning': 'reasoning_12', }
)

preds_gpt_1314 = pd.read_json(
    'results_gpt-4o-mini_13-14.json'
).set_index('ID_Medio_Crono').rename(
    columns={'attack': 'prediction_13', 'attack_reasoning': 'reasoning_13',
             'type_of_attack': 'prediction_14', 'type_of_attack_reasoning': 'reasoning_14', }
)

gpt = preds_gpt_9.join(preds_gpt_1112).join(preds_gpt_1314)

gpt = upv[[
    '¿Qué campaña?',
    '9. Protagonista', '10. Atributo',
    '11. Macro tema', '12. Populismo',
    '13. Ataque', '14. Tipo de ataque']].copy().join(gpt)
# label_preds = label_preds[label_preds['prediction'].notna()]


gpt

,¿Qué campaña?,9. Protagonista,10. Atributo,11. Macro tema,12. Populismo,13. Ataque,14. Tipo de ataque,prediction_9,reasoning_9,prediction_11,reasoning_11,prediction_12,reasoning_12,prediction_13,reasoning_13,prediction_14,reasoning_14
ID_Medio_Crono,,,,,,,,,,,,,,,,,
429,Gen19,"Gobiernos, partidos y dirigentes de derecha",Negativo,Campaign issues,Crítica a las élites,Sí,Electoral,"Gobiernos, partidos y dirigentes de derecha",The article discusses a statement made by José...,Political issues,The article discusses the political situation ...,Crítica a las élites,The article highlights an incorrect assertion ...,Sí,The article describes a situation where a poli...,Ideológico,The nature of the attack focuses on ideologica...
258,Gen19,"Gobiernos, partidos y dirigentes de izquierda",Negativo,Campaign issues,Defensa o apelación al pueblo,Sí,Antecedentes,"Gobiernos, partidos y dirigentes de izquierda",El artículo se centra principalmente en el deb...,Political issues,The article primarily discusses a political de...,Crítica a las élites,Pedro Sánchez's comments criticizing Pablo Igl...,Sí,El artículo describe un ataque verbal del cand...,De gestión,La acusación de Sánchez se centra en la gestió...
265,Gen19,"Gobiernos, partidos y dirigentes de derecha",Negativo,Campaign issues,Ostracismo,Sí,Asociación,"Gobiernos, partidos y dirigentes de derecha",El artículo habla sobre la coalición de gobier...,Political issues,The article primarily addresses the political ...,Crítica a las élites,The fact-checking article critiques a politica...,Sí,The article reviews a statement made by María ...,De gestión,The focus of the article is on the governing a...
365,Gen19,"Gobiernos, partidos y dirigentes de derecha",Negativo,Political issues,Crítica a las élites,Sí,Asociación,"Gobiernos, partidos y dirigentes de derecha",The article discusses the Partido Popular (PP)...,Political issues,The article addresses a specific claim made du...,Crítica a las élites,The statement by Pedro Sánchez is framed as be...,Sí,The article addresses a statement made by Pedr...,De gestión,The attack focuses on the management of govern...
377,Gen19,"Gobiernos, partidos y dirigentes de izquierda",Negativo,Political issues,Crítica a las élites,Sí,Antecedentes,"Gobiernos, partidos y dirigentes de izquierda",The article discusses statements made by Pablo...,Political issues,The article focuses on statements made by Pabl...,Crítica a las élites,Pablo Casado's claims are an example of critic...,Sí,El artículo responde a una afirmación hecha po...,De gestión,El ataque se centra en cuestionar la gestión d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6355,And22,"Gobiernos, partidos y dirigentes de derecha",Negativo,Campaign issues,Crítica a las élites,Sí,Electoral,"Gobiernos, partidos y dirigentes de derecha",The fact-checking article discusses a false cl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6358,And22,"Gobiernos, partidos y dirigentes de derecha",Negativo,Campaign issues,Crítica a las élites,Sí,Electoral,"Gobiernos, partidos y dirigentes de derecha",The protagonist of the article is Juanma Moren...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6360,And22,Personas migrantes,Negativo,Policy issues,Ostracismo,No,Sin ataque,Personas migrantes,The article refutes a false claim that pertain...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# compute results
from sklearn.metrics import f1_score, accuracy_score

predicted_vars = [col.split('_')[-1] for col in gpt.columns
                  if col.startswith('prediction')]

resultsGPT = []
for v in predicted_vars:
    # find column names
    feat = next(col for col in gpt if col.startswith(v))
    pred_col = f'prediction_{v}'
    for campaña, g in gpt.groupby('¿Qué campaña?'):
        # available_preds = g[pred_col].isna().value_counts()
        # display(campaña, available_preds)
        g_ = g[g[feat].notna() & g[pred_col].notna()]
        completion = g_.shape[0]/g.shape[0]

        f1 = f1_score(y_true=g_[feat], y_pred=g_[pred_col], average='macro')
        accuracy = accuracy_score(y_true=g_[feat], y_pred=g_[pred_col])

        resultsGPT.append(
            {'feat': feat, "test": campaña,
             "f1": f1, "accuracy": accuracy,
             "clf": "gpt-4o-mini",
             "completion": format(completion, ".0%")}
        )
resultsGPT = pd.DataFrame(resultsGPT)

# output results
resultsGPT.to_json("results/results_GPT.json", orient="records", indent=4)

In [15]:
# compute baselines
from sklearn.metrics import f1_score, accuracy_score

baselines = []
for feat in upv_features:
    df = upv[['¿Qué campaña?', feat]][upv[feat].notna()].copy()

    for camp, g in df.groupby('¿Qué campaña?'):

        # display(g[feat].mode().item())
        # display(g)
        # break

        try:
            g['mode'] = g[feat].mode()[0]

            f1 = f1_score(y_true=g[feat], y_pred=g['mode'], average='macro')
            accuracy = accuracy_score(y_true=g[feat], y_pred=g['mode'])

            baselines.append(
                {
                    'feat': feat, 'test': camp,
                    'f1': f1, "accuracy": accuracy,
                    'clf': "baseline"
                }
            )
        except Exception as e:
            print(feat, camp)
            debug = g[feat].mode()
            display(g)


baselines = pd.DataFrame(baselines)

# output baselines
baselines.to_json("results/baselines.json", orient="records", indent=4)